<div style="text-align:center; margin-top: 1cm;">
    <h2>Київський політехнічний інститут імені Ігоря Сікорського</h2>
    <h2>Приладобудівний факультет</h2>
    <h3>Кафедра автоматизації та систем неруйнівного контролю</h3>
    <br><br><br>
</div>

<div style="text-align:center; margin-top: 5cm;">
    <h2>Лабораторна робота №3 </h2>
    <h2>з предмету "Комп'ютерний зір"</h2>
</div>

<div style="text-align:right; margin-top: 5cm;">
<p>Студент: Погорєлов Богдан<br>
    Група: ПК-51мп<br>
</p>
</div>
<div style="text-align:center; margin-top: 5cm;">
2025 рік  <br><br><br><br>
</div>

Розробити програму, яка виконує такі дії з ЦВ:
1. Програма зчитує цифрове відео з зовнішньої цифрової камери. Також ця програма розпізнає об’єкти в ЦВ
використовуючи аналіз контурів об’єктів. Програма повинна застосувати мінімум одну геометричну
ознаку, яку визначає розробник програми. Програма формує та відображає вихідне ЦВ, яке містить
контури об’єктів, описані прямокутники та текстові рядки з інформацією про розпізнані об’єкти. При цьому
програма знаходить об’єкт заданої форми (трикутних, чотирикутник і т.п.) з найбільшою площею та
наводить на його геометричний центр оптичну ось цифрової камери. Для цього програма подає відповідні
команди на контролер поворотних платформ з двома серво-приводами.
2. У контролер поворотних платформ записується програма з лабораторної роботи № 2